# Fallen Trees

In [2]:
import os
import shutil
import zipfile
import random

src_images = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen-trees-dataset-1\train\images"
src_labels = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen-trees-dataset-1\train\labels"
dst_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\dataset"  # Temp working directory
zip_path = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\final_dataset.zip"

dst_images = os.path.join(dst_base, "images")
dst_labels = os.path.join(dst_base, "labels")

if not os.path.exists(dst_images):
    shutil.copytree(src_images, dst_images)
if not os.path.exists(dst_labels):
    shutil.copytree(src_labels, dst_labels)

# Change labels 0 to 4
for filename in os.listdir(dst_labels):
    if filename.endswith(".txt"):
        file_path = os.path.join(dst_labels, filename)
        with open(file_path, "r") as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == "0":
                parts[0] = "4"
            new_lines.append(" ".join(parts) + "\n")
        with open(file_path, "w") as f:
            f.writelines(new_lines)

# Split into train/test/valid
image_files = {os.path.splitext(f)[0] for f in os.listdir(dst_images)}
label_files = {os.path.splitext(f)[0] for f in os.listdir(dst_labels)}
common_files = sorted(image_files & label_files)
random.seed(42)
random.shuffle(common_files)

total = len(common_files)
train_count = int(total * 0.7)
test_count = int(total * 0.2)
train_files = common_files[:train_count]
test_files = common_files[train_count:train_count + test_count]
valid_files = common_files[train_count + test_count:]

for split in ["train", "test", "valid"]:
    os.makedirs(os.path.join(dst_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(dst_base, split, "labels"), exist_ok=True)

def move_files(file_list, split):
    for name in file_list:
        img_src = os.path.join(dst_images, name + ".jpg")
        if not os.path.exists(img_src):
            img_src = os.path.join(dst_images, name + ".png")
        lbl_src = os.path.join(dst_labels, name + ".txt")
        shutil.copy(img_src, os.path.join(dst_base, split, "images", os.path.basename(img_src)))
        shutil.copy(lbl_src, os.path.join(dst_base, split, "labels", os.path.basename(lbl_src)))

move_files(train_files, "train")
move_files(test_files, "test")
move_files(valid_files, "valid")

os.makedirs(os.path.dirname(zip_path), exist_ok=True)
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(dst_base):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, dst_base)
            zipf.write(file_path, arcname)

print(f"Final dataset zipped at: {zip_path}")


Final dataset zipped at: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\final_dataset.zip


In [3]:
import os
import shutil
import zipfile
import sys

src_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen-trees-dataset-2"
dst_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\dataset2"
zip_dir  = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees"
zip_path = os.path.join(zip_dir, "final_dataset2.zip")

if not os.path.exists(src_base):
    print(f"Source folder does not exist: {src_base}")
    sys.exit(1)

if not os.path.exists(dst_base):
    print(f"Copying dataset from\n  {src_base}\nto\n  {dst_base} ...")
    shutil.copytree(src_base, dst_base)
else:
    print(f"Working copy already exists, skipping copy: {dst_base}")

label_dirs = [
    os.path.join(dst_base, "train", "labels"),
    os.path.join(dst_base, "valid", "labels"),
]

for label_dir in label_dirs:
    if not os.path.isdir(label_dir):
        print(f"Warning: label directory not found, skipping: {label_dir}")
        continue

    print(f"Processing labels in: {label_dir}")
    for filename in os.listdir(label_dir):
        if not filename.lower().endswith(".txt"):
            continue
        file_path = os.path.join(label_dir, filename)
        try:
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                lines = f.readlines()
        except Exception as e:
            print(f"  Could not read {file_path}: {e}")
            continue

        new_lines = []
        changed = False
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            if parts[0] == "0":
                parts[0] = "4"
                changed = True
            new_lines.append(" ".join(parts) + "\n")

        if changed:
            try:
                with open(file_path, "w", encoding="utf-8") as f:
                    f.writelines(new_lines)
            except Exception as e:
                print(f"  Could not write {file_path}: {e}")
        else:
            with open(file_path, "w", encoding="utf-8") as f:
                f.writelines(new_lines)

print("Label 0 to 4 change completed.")

os.makedirs(zip_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(dst_base):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, dst_base)
            zipf.write(file_path, arcname)

print(f"Modified dataset saved as: {zip_path}")

Copying dataset from
  D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen-trees-dataset-2
to
  D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\dataset2 ...
Processing labels in: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\dataset2\train\labels
Processing labels in: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\dataset2\valid\labels
Label 0 to 4 change completed.
Modified dataset saved as: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\fallen_trees\final_dataset2.zip


# Damaged electrical poles and wires

In [ ]:
import os
import shutil
import zipfile
import sys

src_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles-dataset-1"
dst_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\dataset"
zip_dir  = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles"
zip_path = os.path.join(zip_dir, "final_dataset.zip")

if not os.path.exists(src_base):
    print(f"Source folder not found: {src_base}")
    sys.exit(1)

if not os.path.exists(dst_base):
    print(f"Copying dataset from\n  {src_base}\nto\n  {dst_base} ...")
    shutil.copytree(src_base, dst_base)
else:
    print(f"Working copy already exists: {dst_base}")

label_dirs = [
    os.path.join(dst_base, "train", "labels"),
    os.path.join(dst_base, "test", "labels"),
    os.path.join(dst_base, "valid", "labels"),
]

for label_dir in label_dirs:
    if not os.path.isdir(label_dir):
        print(f"Skipping missing directory: {label_dir}")
        continue

    print(f"Processing label directory: {label_dir}")
    for filename in os.listdir(label_dir):
        if not filename.lower().endswith(".txt"):
            continue

        file_path = os.path.join(label_dir, filename)
        
        if not os.path.isfile(file_path):
            print(f"Label file missing, skipping: {file_path}")
            continue
        
        try:
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                lines = f.readlines()
        except Exception as e:
            print(f"Could not read label file {file_path}: {e}")
            continue

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == "0":
                parts[0] = "9"
            new_lines.append(" ".join(parts) + "\n")

        try:
            with open(file_path, "w", encoding="utf-8") as f:
                f.writelines(new_lines)
        except Exception as e:
            print(f"Could not write label file {file_path}: {e}")

print("Label 0 to 9 change completed.")

os.makedirs(zip_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(dst_base):
        for file in files:
            file_path = os.path.join(root, file)
            if not os.path.isfile(file_path):
                print(f"Skipping missing file while zipping: {file_path}")
                continue
            arcname = os.path.relpath(file_path, dst_base)
            try:
                zipf.write(file_path, arcname)
            except Exception as e:
                print(f"Error adding file to zip: {file_path}: {e}")

print(f"Modified dataset saved as: {zip_path}")

Working copy already exists: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\dataset
Processing label directory: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\dataset\train\labels
Processing label directory: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\dataset\test\labels
Processing label directory: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\dataset\valid\labels
Label 0 to 9 change completed.
Modified dataset saved as: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\final_dataset.zip


In [ ]:
import os
import shutil
import zipfile

src_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles-dataset-2"
dst_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles\dataset2"

if not os.path.exists(dst_base):
    print(f"Copying dataset from\n  {src_base}\nto\n  {dst_base} ...")
    shutil.copytree(src_base, dst_base)
else:
    print(f"Working copy already exists: {dst_base}")

label_dirs = [
    os.path.join(dst_base, "train", "labels"),
    os.path.join(dst_base, "test", "labels"),
    os.path.join(dst_base, "valid", "labels")
]

for label_dir in label_dirs:
    if not os.path.isdir(label_dir):
        print(f"Skipping missing label directory: {label_dir}")
        continue

    print(f"Processing label directory: {label_dir}")
    for filename in os.listdir(label_dir):
        if not filename.lower().endswith(".txt"):
            continue

        file_path = os.path.join(label_dir, filename)
        if not os.path.isfile(file_path):
            print(f"Label file missing, skipping: {file_path}")
            continue

        try:
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                lines = f.readlines()
        except Exception as e:
            print(f"Could not read label file {file_path}: {e}")
            continue

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            if parts[0] == "3":
                # skip label '3' lines entirely
                continue
            if parts[0] == "0":
                parts[0] = "9"
            new_lines.append(" ".join(parts) + "\n")

        try:
            with open(file_path, "w", encoding="utf-8") as f:
                f.writelines(new_lines)
        except Exception as e:
            print(f"Could not write label file {file_path}: {e}")

print("Label 0 to 9 change completed and label 3 removed.")

zip_dir = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles"
os.makedirs(zip_dir, exist_ok=True)
zip_path = os.path.join(zip_dir, "final_dataset2.zip")

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(dst_base):
        for file in files:
            file_path = os.path.join(root, file)
            if not os.path.isfile(file_path):
                print(f"Skipping missing file while zipping: {file_path}")
                continue
            arcname = os.path.relpath(file_path, dst_base)
            try:
                zipf.write(file_path, arcname)
            except Exception as e:
                print(f"Error adding file to zip: {file_path}: {e}")

print(f"Modified dataset saved as: {zip_path}")

Working copy already exists: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles\dataset2
Processing label directory: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles\dataset2\train\labels
Processing label directory: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles\dataset2\test\labels
Processing label directory: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical-poles\dataset2\valid\labels
Label 0 to 9 change completed and label 3 removed.
Modified dataset saved as: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\electrical_poles\final_dataset2.zip


# Graffiti

In [10]:
import os
import shutil
import zipfile

src_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\graffiti-dataset-1"
dst_base = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\graffiti\dataset"

if not os.path.exists(dst_base):
    shutil.copytree(src_base, dst_base)

label_dirs = [
    os.path.join(dst_base, "train", "labels"),
    os.path.join(dst_base, "test", "labels"),
    os.path.join(dst_base, "valid", "labels")
]

for label_dir in label_dirs:
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(label_dir, filename)
            with open(file_path, "r") as f:
                lines = f.readlines()
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts:
                    if parts[0] == "0" or parts[0] == 1:
                        parts[0] = "6"
                    new_lines.append(" ".join(parts) + "\n")
            with open(file_path, "w") as f:
                f.writelines(new_lines)

zip_dir = r"D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\graffiti"
os.makedirs(zip_dir, exist_ok=True)
zip_path = os.path.join(zip_dir, "final_dataset.zip")

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(dst_base):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, dst_base)
            zipf.write(file_path, arcname)

print(f"Modified dataset saved as: {zip_path}")

Modified dataset saved as: D:\Projects\Vision-based-Urban-Issues-Detection\Datasets\graffiti\final_dataset.zip
